**补充涉及计算的函数值**

In [ ]:
%reset -f

In [ ]:
%matplotlib inline
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from utils.iwavelets import xphi0
from utils.aiwt import restrain

In [ ]:
def IncreaseInterp(KnowX,KnowP,NX, J,j0,cjk,cjknum,zJc,zJcnum,a,b,C):
    """
    输入：
        KnowX : ndarray(1, m) -- 上一次迭代得到空间中存在配点的位置集合
        KnowP : ndarray(1, m) -- 上一次迭代得到空间中存在配点的位置对应的函数值集合
        NX : ndarray(n,) -- 此次迭代时得到空间中存在配点的位置集合
        J : int -- 插值小波层数
        j0 : int -- 初始层数
        cjk : list[ndarray(-1), ...] -- 上一次迭代的配点位置索引
        cjknum : list -- 上一次迭代层配点索引数量（每层需计算的配点数量）
        zJc : list -- 上一次迭代配点实际位置
        zJcnum : int -- 上一次迭代实际位置的数量
        a, b : float -- 函数的限制范围
        C : list[ndarray()] -- 多尺度插值算子
    输出：
        P : ndarray(1, n) -- 当前空间中存在配点的位置对应的函数值集合
    """
    # 两自变量点集之间一样的直接赋值
    _, NX_index, KnowX_index = np.intersect1d(NX, KnowX, return_indices=True)
    P = np.zeros_like(NX)
    P[NX_index] = KnowP[0][KnowX_index]
    # 两自变量点集之间不一样的，计算该位置下在当前空间中对应的函数值
    diff = np.setdiff1d(NX, KnowX)
    index = np.where(np.in1d(NX, diff))
    i = np.arange(zJcnum)
    x = NX[index].reshape((1,-1))
    if x.any():
        value = Ii(i,J,j0,cjk,cjknum,zJc,x,a,b,C)
        s = np.dot(KnowP, value.T)
    else:
        s = []

    P[index] = s
    P = P.reshape((-1,1))
    return P


def Ii(i, J, j0, cjk, cjknum, zJc, x, a, b, C):
    k = np.arange(0, 2**j0+1).reshape(1,-1)
    s = xphi0(j0,k,a,b,x)
    rr = restrain(j0,k.reshape(-1),J,zJc)
    value = np.dot(s, rr)
    for j in range(j0, len(C)+1):
        if j > j0 and cjknum[j] == 0:
            break
        if cjknum[j] < 1:
            continue
        k = cjk[j].reshape(1,-1)
        s = xphi0(j+1,2*k+1,a,b,x)
        value = value + np.dot(s, C[j-1])
    return value